# DualVideoMotionTracker

## Packages

In [1]:
from DualVideoMotionTracker import process_videos

## Test

Define the paths for the dorsal and lateral videos, as well as the output CSV file. Run the `process_videos()` function with these paths and the initial frames for each video. Interact with the interface to mark points of interest in both videos and use the bottom buttons for detection options, feeding status, or record deletion. Upon completion, you will receive a CSV file with the coordinates of the points marked in each frame. To finish the process, press ESC.

In [3]:
# Paths to the video files
video_path_dorsal = "videos/6-top.mp4"
video_path_lateral = "videos/6-lat.mp4"

# Output Excel file path
output_excel_path = "data/output_points.csv"

# Process the videos starting from specific frames
process_videos(video_path_dorsal, video_path_lateral, output_excel_path, start_frame_top=1, start_frame_lat=0)

Left Point 1: (0, 0)
Left Point 2: (0, 0)
Left Point 3: (0, 0)
Right Point 1: (0, 0)
Right Point 2: (0, 0)
Right Point 3: (0, 0)
Clicked points for the current frame: [(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
Left Point 1: (270, 118)
Left Point 2: (291, 218)
Left Point 3: (440, 251)
Right Point 1: (199, 97)
Right Point 2: (-2, -2)
Right Point 3: (-2, -2)
Clicked points for the current frame: [(270, 118), (291, 218), (440, 251), (199, 97), (-2, -2), (-2, -2)]
Right Point 1: (482, 105)
Data saved to data/output_points.csv
